# Structural Sensitivity of S&P 500 Drawdowns to NVDA Exposure Assumptions

## Executive Summary

This study evaluates the structural sensitivity of S&P 500 (SPX) drawdown behavior to assumed NVDA index exposure within a simplified linear return decomposition framework. Rather than attempting causal attribution, the analysis isolates how varying structural exposure assumptions affect simulated stress outcomes.

Key findings:

* **Rolling beta analysis**: NVDA has frequently exhibited a systemic-level statistical relationship with SPX returns, with elevated rolling beta regimes observed during early 2020 and again throughout 2025. These periods suggest episodes in which NVDA’s return behavior was unusually influential relative to the broader index.

* **Shock simulations**: Under one-day NVDA shock scenarios, maximum drawdown in SPX scales predictably with both the assumed NVDA weight and the magnitude of the shock. Recovery time remains short in most simulated cases unless both the weight and shock magnitude are extreme.

* **Sensitivity to exposure assumptions**: Heatmap analysis demonstrates that stress outcomes are highly sensitive to structural exposure assumptions. Even under a stylized linear model, larger assumed NVDA weights materially increase simulated index drawdowns.

This framework is intentionally simplified; it assumes constant exposure weights, no endogenous feedback effects, and no multi-day volatility dynamics. As such, results should be interpreted as sensitivity analysis under a stylized structural model rather than as forecasts or causal claims.

Future extensions could incorporate time-varying index weights, multi-day shock paths, and more realistic feedback mechanisms to better approximate market dynamics.

## Motivation and Framing

Recent market commentary has highlighted the increasing concentration of major U.S. equity indices in a small number of large-cap technology firms, particularly during the period of rapid AI-driven valuation expansion. NVIDIA (NVDA), as a leading semiconductor firm closely associated with AI infrastructure growth, has been one of the most prominent contributors to index-level performance in recent years.

While public discussion often focuses on whether such firms are “driving the market,” that framing can conflate statistical co-movement with structural exposure. This study adopts a more mechanical perspective: rather than attempting to estimate causal influence, it evaluates how sensitive index-level stress outcomes are to assumptions about a dominant constituent’s exposure weight.

Specifically, the analysis asks:

> Under a simplified linear decomposition of SPX returns, how do simulated drawdowns respond to different assumptions about NVDA’s structural index weight?

NVDA is therefore used as an illustrative case study within a broader methodological question: how does index-level risk behavior depend on structural exposure assumptions in a cap-weighted framework?

The objective is not to determine whether NVDA “causes” index movements, but to examine how varying exposure assumptions alter stress-test outcomes under stylized conditions.

## Data and Preprocessing

Daily adjusted closing price data for NVDA and the S&P 500 index (SPX) were obtained via the `yfinance` API. Adjusted prices were used to account for corporate actions such as stock splits and dividends, ensuring consistency in return calculations.

Returns were computed as **daily simple returns**, defined as:

$$ r_t = \frac{P_t - P_{t-1}}{P_{t-1}}. $$

Daily simple returns were chosen for interpretability and because, at daily frequency, simple and log returns are numerically similar. Log returns may offer desirable aggregation properties over longer horizons, but for the short-horizon stress simulations conducted here, simple returns provide a sufficiently accurate and transparent representation.

Missing observations were not forward-filled; instead, any dates with incomplete data were excluded to avoid introducing artificial smoothing or bias into the return series.

All subsequent analysis is performed on aligned daily return series for NVDA and SPX over the selected sample period (since the start of 2018), which covers both pre- and post-AI expansion phases as well as the early COVID-19 period.

## Linear Decomposition Framework

To evaluate structural sensitivity, SPX returns are decomposed using a simplified linear exposure identity:

$$ r_{SPX,t} = w r_{NVDA,t} + (1-w)r_{exNVDA,t},$$

where:

* $r_{SPX,t}$ is the daily SPX return,
* $r_{NVDA,t}$ is the daily NVDA return,
* $r_{exNVDA,t}$ represents the return of the index excluding NVDA, and
* $w \in [0,1)$ is an assumed structural weight representing NVDA’s share of index exposure.

Solving algebraically for the residual component yields:

$$ r_{exNVDA,t} = \frac{r_{SPX,t} - w r_{NVDA,t}}{1-w}. $$

This decomposition is mechanical and does not imply causality. The parameter $w$ should not be interpreted as a regression coefficient, beta estimate, or correlation measure. Instead, it represents a structural exposure assumption: the fraction of index return mechanically attributed to NVDA within a cap-weighted framework.

Varying $w$ therefore does not change the observed data, but alters the implied counterfactual index return stream excluding NVDA. Sensitivity analysis over $w$ evaluates how stress outcomes depend on these structural exposure assumptions.

In this study, $w$ is treated as an uncertain parameter and varied over a plausible range to assess robustness of simulated drawdown behavior.